<a href="https://colab.research.google.com/github/mk0653/untitled/blob/master/Accident%20type%20classification/Accident_type_classification_ensamble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

テストデータの読み込み
学習済みモデルの読み込み
各モデルで推論
推論結果の結合
lightboostでスタッキング


In [ ]:
import os
import sys
import re

from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install fugashi[unidic-lite]
!pip install ipadic
!pip install sentencepiece

     |████████████████████████████████| 2.9 MB 2.8 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 490 kB 2.1 MB/s 
     |████████████████████████████████| 47.4 MB 1.3 MB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=1b68f6919eeb4f435decd4126e3387ed19f52defb78bc9996514498a8e0d1b0e
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite
     |████████████████████████████████| 13.4 MB 4.3 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=aa

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import re
import json
import glob
from collections import defaultdict
from textblob import TextBlob
from functools import partial

import pandas as pd
import numpy as np

import torch
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import random

import nltk
import spacy
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS

from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification,RobertaTokenizerFast,RobertaForSequenceClassification,T5TokenizerFast, AutoModelForSequenceClassification,AlbertTokenizerFast, AlbertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import string

%matplotlib inline

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/事故の型分類/train.csv")
valid =pd.read_csv("/content/drive/MyDrive/事故の型分類/dev.csv")

In [ ]:
train_texts = train["sentence"].tolist()
valid_texts = valid["sentence"].tolist()
train_labels = train["label"].tolist()
valid_labels = valid["label"].tolist()

In [ ]:
import re
import os
import urllib.request
import unicodedata
import nltk

from bs4 import BeautifulSoup

#記号の除去
def clean_text(text):
    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(
        r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    return replaced_text

In [ ]:
train_texts_clean = []
valid_texts_clean = []
for x in tqdm(range(len(train_texts))):
  train_texts_clean.append(clean_text(train_texts[x]))

for x in tqdm(range(len(valid_texts))):
  valid_texts_clean.append(clean_text(valid_texts[x]))

train_texts = train_texts_clean
valid_texts = valid_texts_clean 

  0%|          | 0/30109 [00:00<?, ?it/s]

  0%|          | 0/7528 [00:00<?, ?it/s]

In [ ]:
#学習済みモデルの読み込み
model_bert = "/content/drive/MyDrive/bert_large_checkpoint-5750"
model_roberta = "/content/drive/MyDrive/roberta_checkpoint-7500"
model_albert = "/content/drive/MyDrive/albert_checkpoint-5750"
# max sequence length for each document/sentence sample
max_length = 256

#事前学習モデルからトークナイザーの読み込み
tokenizer_bert = BertTokenizerFast.from_pretrained("/content/drive/MyDrive/bert-large", do_lower_case=True)
tokenizer_roberta = T5TokenizerFast.from_pretrained("/content/drive/MyDrive/roberta-base", do_lower_case=True)
tokenizer_albert = AlbertTokenizerFast.from_pretrained("/content/drive/MyDrive/albert-base", do_lower_case=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [ ]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset

def get_data(tokenizer):
  train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
  valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

  train_dataset = NewsGroupsDataset(train_encodings, train_labels)
  valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

  return train_dataset,valid_dataset

In [ ]:
import torch
import torch.nn as nn
from transformers import BertJapaneseTokenizer, AutoModelForSequenceClassification

"""
def pred(model_name,tokenizer,data):
  predictions = []

  model =  AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=train['label'].nunique())
  model.to(device)
  model.eval()
  preds = []
  m= nn.Softmax(dim=0)
  for i in tqdm(range(len(data))):
      tokenized_text = tokenizer.tokenize(data[i])
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
      tokens_tensor = torch.tensor([indexed_tokens])
      with torch.no_grad():
          y_preds = model(tokens_tensor.to(device))[0]
      preds.append(y_preds.argmax(-1).item())
  return preds
"""

def pred(model_name,token,text):
  train_dataset, valid_dataset = get_data(token)
  model =  AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=train['label'].nunique())
  trainer = Trainer(model=model)
  if text == "train":
    predictions1 = trainer.predict(train_dataset)
  else:
    predictions1 = trainer.predict(valid_dataset)
  a = predictions1.predictions
  
  preds = []
  for i in tqdm(range(len(a))):
    preds.append(a[i].tolist().index(max(a[i].tolist())))

  return preds

In [ ]:
pred1 = pred(model_bert,tokenizer_bert,"train")
pred2 = pred(model_roberta,tokenizer_roberta,"train")
pred3 = pred(model_albert,tokenizer_albert,"train")

***** Running Prediction *****
  Num examples = 30109
  Batch size = 8


  0%|          | 0/30109 [00:00<?, ?it/s]

loading configuration file /content/drive/MyDrive/roberta_checkpoint-7500/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    

  0%|          | 0/30109 [00:00<?, ?it/s]

loading configuration file /content/drive/MyDrive/albert_checkpoint-5750/config.json
Model config AlbertConfig {
  "_name_or_path": "/content/drive/MyDrive/albert-base",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "eos_token_ids": null,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_ra

  0%|          | 0/30109 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame({'albert': pred3,
                   'roberta': pred2,
                    'bert': pred1
                   },
)

In [ ]:
kata = {0:"墜落、転落",
        1:"転倒", 
        2:"激突", 
        3:"飛来、落下", 
        4:"崩壊、倒壊",
        5:"激突され",
        6:"はさまれ、巻き込まれ",
        7:"切れ、こすれ",
        8:"踏み抜き",
        9:"おぼれ",
        10:"高温・低温の物との接触",
        11:"有害物等との接触",
        12:"感電",
        13:"爆発",
        14:"破裂",
        15:"火災",
        16:"交通事故（道路）",
        17:"交通事故（その他）",
        18:"動作の反動、無理な動作",
        19:"その他",
        20:"分類不能",
        }

In [ ]:
from sklearn.metrics import accuracy_score,fbeta_score

fbeta = fbeta_score(train_labels , pred2,average=None, beta=0.5)
n = 0
for x in fbeta:
  print(kata[n],x)
  n+=1

墜落、転落 0.973622463450481
転倒 0.8553719008264463
激突 0.43032786885245905
飛来、落下 0.8535844471445929
崩壊、倒壊 0.9064614712944773
激突され 0.778023598820059
はさまれ、巻き込まれ 0.9127756160830093
切れ、こすれ 0.8344459279038717
踏み抜き 0.0
おぼれ 0.898876404494382
高温・低温の物との接触 0.8998895841001102
有害物等との接触 0.9582036460649178
感電 0.9838403041825096
爆発 0.9533267130089375
破裂 0.9074074074074073
火災 0.9187158469945357
交通事故（道路） 0.9806927306357209
交通事故（その他） 0.8944954128440366
動作の反動、無理な動作 0.0
その他 0.8914473684210527
分類不能 0.0


In [ ]:
valid_pred1 = pred(model_bert,tokenizer_bert,"valid")
valid_pred2 = pred(model_roberta,tokenizer_roberta,"valid")
valid_pred3 = pred(model_albert,tokenizer_albert,"valid")

loading configuration file /content/drive/MyDrive/bert_large_checkpoint-5750/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/bert-large/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LAB

  0%|          | 0/7528 [00:00<?, ?it/s]

loading configuration file /content/drive/MyDrive/roberta_checkpoint-7500/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    

  0%|          | 0/7528 [00:00<?, ?it/s]

loading configuration file /content/drive/MyDrive/albert_checkpoint-5750/config.json
Model config AlbertConfig {
  "_name_or_path": "/content/drive/MyDrive/albert-base",
  "architectures": [
    "AlbertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "eos_token_ids": null,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_ra

  0%|          | 0/7528 [00:00<?, ?it/s]

In [ ]:
fbeta = fbeta_score(valid_labels , valid_pred1,average="micro", beta=0.5)
fbeta

0.8687566418703507

In [ ]:
df_valid = pd.DataFrame({
                   'albert': valid_pred3,
                   'roberta': valid_pred2,
                   'bert': valid_pred1},
)

LightGBMによるスタッキング

In [ ]:
import lightgbm as lgb
train_set = lgb.Dataset(df, label = train_labels)
valid_set = lgb.Dataset(df_valid, label = valid_labels)

params = {
    "objective" : "multiclass",
    "metric" : "multi_logloss",
    "num_class" : 21,
    "learning_rate":5e-2,
    "num_leaves":50,
    "max_depth":5
}

result_data = {}
model = lgb.train(
    params = params,
    train_set = train_set,
    valid_sets = [train_set, valid_set],
    num_boost_round = 1000,
    early_stopping_rounds = 1,
    verbose_eval = 10,
    evals_result = result_data
)

Training until validation scores don't improve for 1 rounds.
[10]	training's multi_logloss: 1.06755	valid_1's multi_logloss: 1.15829
[20]	training's multi_logloss: 0.719502	valid_1's multi_logloss: 0.853422
[30]	training's multi_logloss: 0.534931	valid_1's multi_logloss: 0.697961
[40]	training's multi_logloss: 0.430016	valid_1's multi_logloss: 0.615479
[50]	training's multi_logloss: 0.368442	valid_1's multi_logloss: 0.57182
[60]	training's multi_logloss: 0.331714	valid_1's multi_logloss: 0.550169
[70]	training's multi_logloss: 0.309394	valid_1's multi_logloss: 0.539923
[80]	training's multi_logloss: 0.295491	valid_1's multi_logloss: 0.53635
[90]	training's multi_logloss: 0.286468	valid_1's multi_logloss: 0.535587
Early stopping, best iteration is:
[90]	training's multi_logloss: 0.286468	valid_1's multi_logloss: 0.535587


In [ ]:
prediction = model.predict(df_valid)
#results = model_selection.cross_val_score(xgbc2, X_test, y_test, cv = kfold) 

In [ ]:
last_preds = []
for i in tqdm(range(len(prediction))):
  last_preds.append(prediction[i].tolist().index(max(prediction[i].tolist())))

  0%|          | 0/7528 [00:00<?, ?it/s]

In [ ]:
kata = {0:"墜落、転落",
        1:"転倒", 
        2:"激突", 
        3:"飛来、落下", 
        4:"崩壊、倒壊",
        5:"激突され",
        6:"はさまれ、巻き込まれ",
        7:"切れ、こすれ",
        8:"踏み抜き",
        9:"おぼれ",
        10:"高温・低温の物との接触",
        11:"有害物等との接触",
        12:"感電",
        13:"爆発",
        14:"破裂",
        15:"火災",
        16:"交通事故（道路）",
        17:"交通事故（その他）",
        18:"動作の反動、無理な動作",
        19:"その他",
        20:"分類不能",
        }

In [ ]:
from sklearn.metrics import accuracy_score,fbeta_score

fbeta = fbeta_score(valid_labels , last_preds ,average=None, beta=0.5)
n = 0
for x in fbeta:
  print(kata[n],x)
  n+=1

墜落、転落 0.9312594079277472
転倒 0.7792887029288703
激突 0.7432432432432431
飛来、落下 0.759845890410959
崩壊、倒壊 0.8089033659066233
激突され 0.6220095693779903
はさまれ、巻き込まれ 0.8633235004916421
切れ、こすれ 0.8247422680412371
踏み抜き 0.0
おぼれ 0.7604895104895105
高温・低温の物との接触 0.5351681957186544
有害物等との接触 0.8818181818181818
感電 0.9762308998302207
爆発 0.9836065573770493
破裂 0.7142857142857143
火災 0.8293838862559241
交通事故（道路） 0.944003733084461
交通事故（その他） 0.8097686375321336
動作の反動、無理な動作 0.642023346303502
その他 0.0


In [ ]:
fbeta = fbeta_score(valid_labels , last_preds,average="micro", beta=0.5)
fbeta

0.8727417640807651

ここから多数決によるアンサンブル

In [ ]:
vosting_pred= df_valid.mode(axis=1)[0].tolist()

In [ ]:
fbeta = fbeta_score(valid_labels , vosting_pred,average="micro", beta=0.5)
fbeta

0.8813761955366631